In [1]:
from uk_covid19 import Cov19API
import json
import pandas as pd
import matplotlib.pyplot as plt
import ipywidgets as wdg
import numpy as np


%matplotlib inline
plt.rcParams['figure.dpi'] = 100


In [2]:
def parse_date(datestring):
    """ Convert a date string into a pandas datetime object """
    return pd.to_datetime(datestring, format="%Y-%m-%d")

THE LATEST COVID DATA FROM PUBLIC HEALTH ENGLAND

NEW CASES, HOSPITAL ADMISSIONS AND DEATHS

In [59]:
def timeseries_api():
    filters = [
        'areaType=overview' 
    ]

    structure = {
        "date": "date",
        "cases": "newCasesByPublishDate",
        "hospital": "newAdmissions",
        "deaths": "cumDeaths28DaysByDeathDateRate"    
    }

    api = Cov19API(filters=filters, structure=structure)
    timeSeries=api.get_json()

    
    with open("timeSeries.json", "wt") as OUTF:
        json.dump(timeSeries, OUTF)
    
    with open("timeSeries.json", "rt") as INFILE:
        timeSeriesData=json.load(INFILE)
    
    return timeSeriesData

def wrangle_data_timeseries(rawdata):
    timeSeriesDataList = rawdata['data']
    
    timeSeriesDates=[dictionary['date'] for dictionary in timeSeriesDataList]
    timeSeriesDates.sort()

    timeSeriesStartDate = parse_date(timeSeriesDates[0])
    timeSeriesEndDate = parse_date(timeSeriesDates[-1])

    index = pd.date_range(timeSeriesStartDate, timeSeriesEndDate, freq ='D')
    timeSeriesDF = pd.DataFrame(index=index, columns = ['cases', 'hospital', 'deaths'])

    for entry in timeSeriesDataList:
        date=parse_date(entry['date'])
        for column in ['cases', 'hospital', 'deaths']:
            if pd.isna(timeSeriesDF.loc[date, column]): 
                value= float(entry[column]) if entry[column]!=None else 0.0
                timeSeriesDF.loc[date, column]=value
    timeSeriesDF.fillna(0.0, inplace=True)  
    return timeSeriesDF

series=wdg.SelectMultiple(
    options=['cases', 'hospital', 'deaths'],
    value=['cases', 'hospital', 'deaths'],
    rows=3,
    description='Stats:',
    disabled=False
    )

scaleTimeseries=wdg.RadioButtons(
    options=['linear', 'log'],
    description='Scale:',
    disabled=False
    )

controls=wdg.HBox([series, scaleTimeseries])

def timeseries_graph(gcols, gscale):
    if gscale=='linear':
        logscale=False
    else:
        logscale=True
    ncols=len(gcols)
    if ncols>0:
        timeSeriesDataFrame[list(gcols)].plot(logy=logscale)
    else:
        print("Click to select data for graph")
        print("(CTRL-Click to select more than one category)")


    
with open("timeSeries.json", "rt") as INFILE:
        timeSeriesData=json.load(INFILE)
        
timeSeriesDataFrame = wrangle_data_timeseries(timeSeriesData);

graph=wdg.interactive_output(timeseries_graph, {'gcols': series, 'gscale': scaleTimeseries})


# stack series and scale on top of each other
ctrls=wdg.HBox([series, scaleTimeseries])

# put the graph and the controls side by side
form=wdg.HBox([graph, ctrls])
# Now form contains the entire interface
# display(form)


display(controls, graph)

def refresh_timeseries(button):
    def refresh_graph():
        current = scaleTimeseries.value
        if current == scaleTimeseries.options[0]:
            other = scaleTimeseries.options[1]
        else:
            other = scaleTimeseries.options[0]
        scaleTimeseries.value = other # forces the redraw
        scaleTimeseries.value = current # now we can change it back   
    
    try:
        rawdata = timeseries_api()
        global timeSeriesDataFrame
        timeSeriesDataFrame = wrangle_data_timeseries(rawdata);
        refresh_graph()
        
        apibuttonSeries.icon="check"
        print('Refresh successful', flush = True)   
    
    except:
        print('Refresh failed', flush = True)    
    
    
apibuttonSeries=wdg.Button(
    description='Refresh data',
    disabled=False,
    button_style='', 
    tooltip='Click to download current Public Health England data',
    icon='download' 
)

apibuttonSeries.on_click(refresh_timeseries)

display(apibuttonSeries)

Output()

Button(description='Refresh data', icon='download', style=ButtonStyle(), tooltip='Click to download current Pu…

linear
('linear', 'log')
Refresh successful
log
('linear', 'log')
Refresh successful


In [62]:
def agedistribution_api():

    filters = [
    'areaType=nation',
    'areaName=England'
    ]


    structure = {
    "males": "maleCases",
    "females": "femaleCases"
    }

    api = Cov19API(filters=filters, structure=structure)
    ageDistribution=api.get_json()

    with open("ageDistribution.json", "wt") as OUTF:
        json.dump(ageDistribution, OUTF)

    with open("ageDistribution.json", "rt") as INFILE:
        ageDistributionData=json.load(INFILE)

    return ageDistributionData




def wrangle_data_agedistribution(rawData):

    ageDistributionDataList = rawData['data']
    
    dataDic=rawData['data'][0]
    
    males=dataDic['males']
    females=dataDic['females']

    ageranges=[x['age'] for x in males] 
   
    def min_age(agerange):

        agerange=agerange.replace('+','') 
        start=agerange.split('_')[0]
        return int(start)

    ageranges.sort(key=min_age)
    ageDataFrame=pd.DataFrame(index=ageranges, columns=['males','females', 'total'])

    for entry in males: 
        ageband=entry['age'] 
        ageDataFrame.loc[ageband, 'males']=entry['value']

    for entry in females:
        ageband=entry['age']
        ageDataFrame.loc[ageband, 'females']=entry['value']

    ageDataFrame['total']=ageDataFrame['males']+ageDataFrame['females']
    return ageDataFrame

    
def age_graph(graphcolumns):
        ncols=len(graphcolumns)
        if ncols>0:
            ageDataFrame.plot(kind='bar', y=list(graphcolumns)) 
        else:
            print("Click to select data for graph")
            print("(CTRL-Click to select more than one category)")    

agecols = wdg.SelectMultiple(
    options=['males', 'females', 'total'], 
    value=['males','females', 'total'],
    rows=3, 
    description='Sex',
    disabled=False
    )


with open("ageDistribution.json", "rt") as INFILE:
       ageDistributionData=json.load(INFILE)
        
ageDataFrame = wrangle_data_agedistribution(ageDistributionData);

graph=wdg.interactive_output(age_graph, {'graphcolumns': agecols})

display(agecols, graph)

def refresh_agedistribution(button):
    def refresh_graph():
        current = agecols.value
        print('current', current)
        if current == (agecols.options[0],agecols.options[1]):
            other = (agecols.options[1],agecols.options[2])
        else:
            other = (agecols.options[0],agecols.options[1])
        agecols.value = other # forces the redraw
        agecols.value = current # now we can change it back   
    
    try:
        rawData = agedistribution_api()
        global ageDataFrame
        ageDataFrame = wrangle_data_agedistribution(rawData);
        refresh_graph()
        apibuttonAge.icon="check"
        print('Refresh successful', flush = True)
    
    except:
        print('Refresh failed', flush = True)
    

apibuttonAge=wdg.Button(
    description='Refresh data',
    disabled=False,
    button_style='', 
    tooltip='Click to download current Public Health England data',
    icon='download' 
)

apibuttonAge.on_click(refresh_agedistribution)

display(apibuttonAge)


SelectMultiple(description='Sex', index=(0, 1, 2), options=('males', 'females', 'total'), rows=3, value=('male…

Output()

Button(description='Refresh data', icon='download', style=ButtonStyle(), tooltip='Click to download current Pu…

current ('males',)
Refresh successful


In [48]:
def admissionsbeds_api():
    filters = [
        'areaType=nation',
        'areaName=England'
    ]


    structure = {
        "date":"date",
        "newAdmissions":"newAdmissions",
        "occupiedBeds":"covidOccupiedMVBeds"
    }

    api = Cov19API(filters=filters, structure=structure)
    admissionsBeds=api.get_json()

    with open("admissionsBeds.json", "wt") as OUTF:
        json.dump(admissionsBeds, OUTF)

    with open("admissionsBeds.json", "rt") as INFILE:
        admissionsBedsData=json.load(INFILE)
        
    return admissionsBedsData
        
def wrangle_data_admissionsbeds(rawdata):   
    admissionsBedsDataList = rawdata['data']

    admissionsBedsDates=[dictionary['date'] for dictionary in admissionsBedsDataList]
    admissionsBedsDates.sort()
    admissionsBedsDates

    admissionsBedsStartDate = parse_date(admissionsBedsDates[0])
    admissionsBedsEndDate = parse_date(admissionsBedsDates[-1])

    index = pd.date_range(admissionsBedsStartDate, admissionsBedsEndDate, freq ='D')
    admissionsBedsDataFrame = pd.DataFrame(index=index, columns = ['newAdmissions', 'occupiedBeds'])

    for entry in admissionsBedsDataList:
        date=parse_date(entry['date'])
        for column in ['newAdmissions', 'occupiedBeds']:
            if pd.isna(admissionsBedsDataFrame.loc[date, column]): 
                value= float(entry[column]) if entry[column]!=None else 0.0
                admissionsBedsDataFrame.loc[date, column]=value

    admissionsBedsDataFrame.fillna(0.0, inplace=True)      
    return admissionsBedsDataFrame

def admissions_beds_graph(gcols, gscale):
        if gscale=='linear':
            logscale=False
        else:
            logscale=True
        ncols=len(gcols)
        if ncols>0:
            admissionsBedsDataFrame[list(gcols)].plot(logy=logscale)
        else:
            print("Click to select data for graph")
            print("(CTRL-Click to select more than one category)")


admissions=wdg.SelectMultiple(
    options=['newAdmissions','occupiedBeds'],
    value=['newAdmissions','occupiedBeds'],
    rows=2,
    description='Stats:',
    disabled=False
    )

scale=wdg.RadioButtons(
    options=['linear', 'log'],
    description='Scale:',
    disabled=False
)



with open("admissionsBeds.json", "rt") as INFILE:
    admissionsBedsData=json.load(INFILE)

admissionsBedsDataFrame = wrangle_data_admissionsbeds(admissionsBedsData)

admissionsGraph=wdg.interactive_output(admissions_beds_graph, {'gcols': admissions, 'gscale': scale})

admissionsControls=wdg.HBox([admissions, scale])

display(admissionsControls, admissionsGraph)


def refresh_admissionsbeds(button):
    def refresh_graph():
        current = scale.value
        
        if current == scale.options[0]:
            other = scale.options[1]
        else:
            other = scale.options[0]
        scale.value = other # forces the redraw
        scale.value = current # now we can change it back   
    
    try:
        rawData = admissionsbeds_api()
        global ageDataFrame
        admissionsBedsDataFrame = wrangle_data_admissionsbeds(rawData);
        refresh_graph()
        apibuttonBeds.icon="check"
        print('Refresh successful', flush = True)
    
    except:
        print('Refresh failed', flush = True)
    
    

apibuttonBeds=wdg.Button(
    description='Refresh data',
    disabled=False,
    button_style='', 
    tooltip='Click to download current Public Health England data',
    icon='download' 
)

apibuttonBeds.on_click(refresh_admissionsbeds)

display(apibuttonBeds)


Output()

Button(description='Refresh data', icon='download', style=ButtonStyle(), tooltip='Click to download current Pu…

current linear
options ('linear', 'log')
Refresh successful
current linear
options ('linear', 'log')
Refresh successful
